## Intro
An object is an unordered collection of *properties*,each of which has a *name* and a *value*. Property name is a `String` or a `Symbol`. An object's property can be accessed using a dot notation or square bracket notation. The square bracket notation is more versatile, we can pass an expression which evaluates to string or can be converted to a string or Symbol.

## Creating Objects
### Object Literal Syntax

In [ ]:
let car = {
    model: "911",
    manufacturer: "Porsche",
    year: 2012,    // There can be a trailing comma
}

Recent versions of JavaScript have extended the syntax for object literals in a number of useful ways:  
**Shorthand:** 

In [ ]:
let a = 5, b = 10;
let o = {a, b}  // Equivalent to let 0 = {a: a, b: b}

// Function definition
let p = {hi(){console.log("Hi"}}  // Equivalent to let p = {hi: function(){console.log("Hi")}}

**Computed property name:** Consider the scenario where you are creating an object and property name is not a runtime constant. In older versions of JS, this would be the code:

In [ ]:
const PROPERTY_NAME = "p1";
function computePropertyName() { return "p" + 2; }
let o = {};
o[PROPERTY_NAME] = 1;
o[computePropertyName()] = 2;

In newer version, it can be shortened to:

In [ ]:
const PROPERTY_NAME = "p1";
function computePropertyName() { return "p" + 2; }
let p = {
    [PROPERTY_NAME]: 1,
    [computePropertyName()]: 2
};

**Symbol property name:** 

In [ ]:
const name = Symbol("Property Name");
let p = {
    [name]: "Property value"
}

**Spread Operator:** note that spread operator only spreads the own properties of an object, not any inherited ones

In [ ]:
let position = { x: 0, y: 0 };
let dimensions = { width: 100, height: 75 };
let rect = { ...position, ...dimensions };
let dim = rect.x + rect.y + rect.width + rect.height

// Overriding property
let o = {a: 5}
let p = {a: 6, ...o} // p.a = 5
let q = {...o, a: 6} // q.a = 6, property overridden

### Using new Operator
The `new` keyword must be followed by a function invocation. A function used in this way is called a *constructor* and serves to initialize a newly created object.

In [ ]:
let o = new Object();  // Create an empty object: same as {}.
let a = new Array();   // Create an empty array: same as [].
let d = new Date();    // Create a Date object representing the current time
let r = new Map();     // Create a Map object for key/value mapping

It is common to create one's own constructor:

In [1]:
// Constructor functions start with capital letter by convention
function Person(name, age) {
    this.name = name;
    this.age = age;
    return this;
}

let person = new Person("John Doe", 35);
console.log(person);

Person { name: 'John Doe', age: 35 }


### Using Object.create
Lets us create an object while setting its prototype chain.

## Property Descriptors
Each property of an object has a set of properties associated with it. The following code illustrates the point:

In [2]:
let obj = {
    a: 'Some value'
}

Object.getOwnPropertyDescriptor(obj, 'a')

{
  value: 'Some value',
  writable: true,
  enumerable: true,
  configurable: true
}


**Writable:** set it to `false` to prevent updating its value.

In [ ]:
let obj = {}

Object.defineProperty(obj, 'a', {
    value: 'Can\'t update',
    writable: false
})

obj.a = 'Can update' // fails silently. TypeError in case of strict mode

**Configurable:** set it to `false` to prevent changes to the property descriptor.

In [ ]:
let obj = {}

Object.defineProperty(obj, 'a', {
    value: 'Can\'t update',
    writable: false,
    configurable: false // One way action, can't be reversed
})

// TypeError regardless of strict mode
Object.defineProperty(obj, 'a', {
    writable: true
})

Even after setting configurable to `false`, we can change writable to `false` (though we can't do the reverse).  

**Enumerable:** setting this to `false` prevents the property to show up in `for...in` loop

In [4]:
let obj = {}

Object.defineProperty(obj, 'a', {
    value: 1,
    enumerable: true
})

Object.defineProperty(obj, 'b', {
    value: 2,
    enumerable: false
})

Object.defineProperty(obj, 'c', {
    value: 3,
    enumerable: true
})

// Looping over all enumerable properties owned or up in prototype chain
for(var k in obj){
    console.log('Property ' + k + ' has value ' + obj[k])
}

console.log(Object.keys(obj))  // Does not return non-enumerable properties. Check only own properties.
console.log(Object.getOwnPropertyNames(obj))

Property a has value 1
Property c has value 3
[ 'a', 'c' ]
[ 'a', 'b', 'c' ]


### Immutability
By combining `configurable` as `false` and `writable` as `false`, we can make an object constant. We can also prevent an object from being extended (more properties being added) by using `preventExtensions` static method:

In [ ]:
let obj = {
    a: 1
}

Object.preventExtensions(obj)

obj.b = 2
obj.b // undefined

There are convenience methods to add various degrees of immutability to an object using `Object.seal` and `Object.freeze`. `Object.seal` locks the structure of an object (no adds/deletes), while `Object.freeze` locks both the structure and values (completely read-only).

## Getters and Setters
Whenever we access an object's property, internally a function call `[[Get]]()` occurs. And whenever we set a property, `[[Put]]()` is executed.

In [5]:
let myObject = {
    get a(){
        return this._a_ // _a_ is the actual property holding the value
    },
    
    set a(value){
        this._a_ = value * value
    }
}

myObject.a = 5 // Calls setter
console.log(myObject.a)  // Calls getter

console.log(myObject);

25
{ a: [Getter/Setter], _a_: 25 }


We can also set *getter/setter* using the `defineProperty` method discussed earlier.

In [12]:
let sampleObject = {};
Object.defineProperty(sampleObject, 'PI', {
    get: function(){
        return 3.14
    }
})

sampleObject.PI = 2.0;
sampleObject.PI;

8:14 - Cannot assign to 'PI' because it is a read-only property.


Checking the property descriptor:

In [13]:
Object.getOwnPropertyDescriptor(sampleObject, 'PI');

{
  get: [Function: get],
  set: undefined,
  enumerable: false,
  configurable: false
}


`[[Get]]` operation works in the following way:  
<img src="images/get_operation.jpeg" width=700 height=auto />

`[[Put]]` operation works in the following way:  
<img src="images/put_operation.jpeg" width=750 height=auto />

## `this` Keyword
The `this` keyword is one of the most confusing part of JavaScript. Some examples below to help us understand the confusion

In [15]:
function myCounter(){
    this.count++;
}

myCounter.count = 0;

for(let i = 0; i < 5; i++){
    myCounter();
}

console.log(myCounter.count)

0


In the above example, `this` refers to the global object, which in case of browsers is the `window` object.

The important thing to remember while trying to understand what `this` refers to is to observe the *call-site* rather than the *declaration site*. We look at the 4 rules which will help us understand the use of this.

**Default Binding:** `this` refers to the global object

In [ ]:
function foo(){
    console.log(this.a)
}

var a = 20; // let does not create property on window object
            // If we used let, we would get undefined

foo() // 20

**Implicit Binding:** comes into effect when we call the function in the manner `<object>.<function>()`. In this case, `this` refers to the owning or containing object.

In [17]:
function foo(){
    console.log(this.a)
}

var obj = {
    foo: foo,
    a: 25
}

obj.foo()

25


But we have to be careful in our usage:

In [ ]:
var anotherObj = {
    foo: function(){
        console.log(this.a)
    },
    
    a: 'From anotherObj'
}

var bar = anotherObj.foo // just a reference assignment
var a = 'From outside'

bar() // Notice how bar is being called, default binding in effect. Prints From outside

**Explicit Binding:** using function methods like `call()` and `apply()` we can explicitly state which object `this` should refer to:

In [ ]:
function foo(){
    console.log(this.a)
}

var obj = {
    a: true
}

foo.call(obj)

`call` and `apply` are the same except for how we pass function's argument. We can understand the difference if we look at the syntax:
- `call`: `func.call([thisArg[, arg1, arg2, ...argN]])`
- `apply`: `func.apply(thisArg, [ argsArray])`

If we pass `null` or `undefined` as the first argument, in browsers it is assumed to be the global object.

**Hard Binding:**

In [ ]:
function foo(){
    console.log(this.a)
}

var obj = {
    a: 3.14
}

function bar(){
    foo.call(obj)
}

bar.call(window) // returns 3.14

In the above example, we have hard bound function `foo` to obj by creating a new function `bar`. A more formal way to achieve this is:

In [ ]:
function bind(fn, obj){
    return function(){
        return fn.call(obj, arguments)
    }
}

function foo(){
    console.log(this.a)
}

var obj = {
    a: 3.14
}

foo = bind(foo, obj)
foo() // 3.14

JavaScript already provides such a utility in form of `bind()` method.

In [ ]:
function foo(){
    console.log(this.a)
}

var obj = {
    a: 777
}

var a = 888
foo.bind(obj)()

**new Binding:** when we call a constructor function with `new` keyword, a number of steps occur, most importantly a new object is created and the newly constructed object is set as the `this` binding for that function call.

In [19]:
function foo(a) {
    this.a = a;
}

var obj = new foo(2)
console.log(obj.a)

2


What if mutiple rules apply at the call site? The precedence is:
> New Binding > Explicit Binding > Implicit Binding > Default Binding

In [20]:
// Explicit vs Implicit binding
function foo(){
    console.log(this.a)
}

var obj1 = {
    a: 500,
    foo: foo
}
var obj2 = {
    a: 200,
    foo: foo
}

obj1.foo.call(obj2)
obj2.foo.call(obj1)

200
500


In [21]:
// New vs Implicit binding
function foo(val){
    this.a = val
}

var obj = {
    a: 15,
    foo: foo
}

var bar = new obj.foo(25)

console.log(bar.a)
console.log(obj.a)

25
15


`new` keyword cannot be used together with `call` or `apply`.

In [22]:
// New vs hard binding
function foo(val){
    this.a = val
}

var obj = {
    a: 15
}

var bar = foo.bind(obj)

var baz = new bar(51)

console.log(baz.a)
console.log(obj.a)

51
15


Note that we wouldn't get the same result if we used the `bind` function we created. ES5's `bind` function does some extra work behind the scenes.

### Arrow Function and `this`
Arrow functions do not obey the above defined rules. Instead they rely on lexical scope in which they were defined. For example:

var a = 'Global a'

function foo() {
    var a = 'Inside function'
    
    setTimeout(() => {
        console.log(this.a) // A is inherited from the foo function scope
    }, 100)
}

var obj = {
    a: 'Inside object'
}

foo.call(obj)
foo()

In essence the above code snippet is equivalent to:

In [ ]:
function foo(){
    var a = 'Inside function'
    
    var self = this
    setTimeout(function(){
        console.log(self.a) // A is inherited from the foo function scope
    }, 100)
}

A good example illustrating the difference is event listener callback:

In [ ]:
var button = document.getElementById('testButton')

// In this case "this" refers to the button object
button.addEventListener('click', function(event){
    this.innerHTML = this.innerHTML.toUpperCase()
})

In [ ]:
var button = document.getElementById('testButton')

var innerHTML = 'Global Context'

// In this case "this" refers to the global object
// So on clicking the button innerHTML variable will get capitalised
button.addEventListener('click', (event) => {
    this.innerHTML = this.innerHTML.toUpperCase()
})

## Copying Objects
One way is to use *spread operator* `...` which copies enumerable properties of an object:

In [1]:
let someArray = ['hello', 'bonjour', 'hola'];
let copiedArray = { ...someArray }

copiedArray

{ '0': 'hello', '1': 'bonjour', '2': 'hola' }


We can see that the `length` property of the array is not copied over. This is because it is not *enumerable*. Another thing to note is that an object's property descriptors are not copied over correctly.

In [2]:
let someObject = {}
Object.defineProperty(someObject, 'name', {
    value: 'Lui Kang',
    writable: false,
    enumerable: true,
    configurable: false
})

let anotherObject = { ...someObject }
Object.getOwnPropertyDescriptor(anotherObject, 'name')

{
  value: 'Lui Kang',
  writable: true,
  enumerable: true,
  configurable: true
}


Another thing to note is that the spread operator does *shallow copying*:

In [3]:
let objA = {
    'weight': 5.9,
    'composition': {
        'iron': 0.85,
        'nickel': 0.14,
        'carbon': 0.01
    }
}

let objB = { ...objA }
objA.composition['iron'] = 0.80

console.log(objB.composition['iron'])

0.8


What about objects with getters and setters?

In [ ]:
let objWithGetterAndSetter = {
    get someProperty(){
        return this.p;
    },
    
    set someProperty(value){
        this.p = value;
    }
}

let objectCopy = { ...objWithGetterAndSetter }  // Does not copy getter and setter

`Object.assign` is very similar to spread operator except that it uses assignment to create the properties of the copy:

In [ ]:
let original = {
    get a(){
        return this._a_;
    },
    
    set a(value){
        this._a_ = value;
    }
}

let duplicate = {}

Object.assign(duplicate, original)

We can also copy property descriptors:

In [4]:
let fruit = {}
Object.defineProperties(fruit, {
    name: {
        value: 'Apple',
        writable: false,
        enumerable: true,
        configurable: false
    },
    
    color: {
        value: 'Red',
        writable: true,
        enumerable: true,
        configurable: true
    }
})

let fruitCopy = {}
Object.defineProperties(fruitCopy, Object.getOwnPropertyDescriptors(fruit))
Object.getOwnPropertyDescriptors(fruitCopy)

{
  name: {
    value: 'Apple',
    writable: false,
    enumerable: true,
    configurable: false
  },
  color: {
    value: 'Red',
    writable: true,
    enumerable: true,
    configurable: true
  }
}


This has the advantage of copying `enumerable` properties. It also copies property descriptions faithfully (including getter/setters). But the copy is still a shallow copy.

### Deep Copy